In [ ]:
import numpy as np
import librosa
from optcutfreq import optcutfreq
import os
import random
import pandas as pd
import gc

def generar_audios(archivos, carpeta, frecuencia_muestreo):
    for archivo in archivos:
        ruta_audio = os.path.join(carpeta, archivo)
        audio, sr = librosa.load(ruta_audio, sr=frecuencia_muestreo)
        yield archivo, audio, sr

def calcular_frecuencia_corte_ideal(carpeta, num_audios, frecuencia_muestreo=44100):
    # Obtener el nombre de la carpeta
    nombre_carpeta = os.path.basename(carpeta)

    # Listar los archivos de audio en la carpeta
    archivos = [archivo for archivo in os.listdir(carpeta) if archivo.lower().endswith('.wav')]

    # Verificar si hay suficientes archivos de audio en la carpeta
    if len(archivos) < num_audios:
        print("No hay suficientes archivos de audio en la carpeta.")
        return

    # Seleccionar aleatoriamente 'num_audios' archivos de audio
    archivos_a_procesar = random.sample(archivos, num_audios)

    # Crear un DataFrame vacío para almacenar los resultados
    resultados_df = pd.DataFrame(columns=['Nombre del Audio', 'Frecuencia de Corte Óptima'])

    for archivo, audio, sr in generar_audios(archivos_a_procesar, carpeta, frecuencia_muestreo):
        # Obtener la frecuencia de corte óptima
        fc_opt = optcutfreq(audio, freq=sr, show=True)

        # Agregar los resultados al DataFrame
        resultados_df = resultados_df.append({'Nombre del Audio': archivo, 'Frecuencia de Corte Óptima': fc_opt}, ignore_index=True)

        # Imprimir la frecuencia de corte óptima para cada archivo
        print(f"Frecuencia de corte óptima para {archivo}: {fc_opt}")

        # Liberar memoria eliminando las variables que ya no necesitas
        del audio, sr

        # Recolectar basura para liberar memoria no utilizada
        gc.collect()

    # Guardar el DataFrame en un archivo CSV con el nombre de la carpeta
    nombre_archivo_csv = f'/content/drive/MyDrive/Eco acústica/Zamuro/RZUB04{nombre_carpeta}_cut_off.csv'
    resultados_df.to_csv(nombre_archivo_csv, index=False)

    # Liberar memoria eliminando el DataFrame completo
    del resultados_df

# Ejemplo de uso
carpeta_audios = '/content/drive/MyDrive/Eco acústica/Zamuro/RZUB04'
num_audios_a_analizar = 30
frecuencia_muestreo = 192000

calcular_frecuencia_corte_ideal(carpeta_audios, num_audios_a_analizar, frecuencia_muestreo)

In [ ]:
import pandas as pd
import os

# DataFrame global para acumular los resultados
resultados_df = pd.DataFrame()

def procesar_archivo(ruta_archivo):
    global resultados_df

    # Obtén el nombre del archivo CSV (sin la extensión) para usarlo como nombre de columna
    nombre_archivo = os.path.splitext(os.path.basename(ruta_archivo))[0]

    # Lee el archivo CSV
    df = pd.read_csv(ruta_archivo)

    # Calcula el promedio de la columna "Frecuencia de Corte Óptima"
    promedio = df["Frecuencia de Corte Óptima"].mean()

    # Agrega una columna al DataFrame global con el nombre del archivo y el promedio
    resultados_df[nombre_archivo] = [promedio * 2]

# Ejemplo de uso
archivos = ["/content/drive/MyDrive/Eco acústica/Zamuro/RZUA01RZUA01_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUA04RZUA04_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUB04RZUB04_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUC03RZUC03_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUC10RZUC10_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUD07RZUD07_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUE07RZUE07_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUF09RZUF09_cut_off.csv",
            "/content/drive/MyDrive/Eco acústica/Zamuro/RZUG04RZUG04_cut_off.csv"]

for archivo in archivos:
    procesar_archivo(archivo)

# Guarda el DataFrame global en un archivo CSV
resultados_df.to_csv("/content/drive/MyDrive/Eco acústica/Zamuro/Frecaucia_Media_Zamuro.csv", index=False)